In [1]:
import sqlite3
from pxr import Usd, UsdGeom, Gf, Vt, Sdf

In [2]:
def load_table(db_file, table, cn):
   conn = sqlite3.connect(db_file)
   conn.row_factory = sqlite3.Row
   cursor = conn.cursor()
   query = f'SELECT * FROM {table};'
   cursor.execute(query)
   results = cursor.fetchall()
   columns=results[0].keys()
   print(table,len(results), columns)
   res={}
   for r in results:      
      ins={}
      for c in columns[1:]:
         ins[c]=r[c]
      res[r['id']]=type(cn,(),ins)
   cursor.close()
   conn.close()
   return res

In [3]:
database_file = "geometry_atlas.db"
sh_d = load_table(database_file, 'Shapes','SH')

Shapes 119487 ['id', 'type', 'parameters']


In [5]:
def create_box(id, pars):
    dx=pars['XHalfLength']
    dy=pars['YHalfLength']
    dz=pars['ZHalfLength']
    stage: Usd.Stage = Usd.Stage.CreateNew(f"assets/box_{id}.usda")

    box = UsdGeom.Mesh.Define(stage, f'/box_{id}')
    box.CreatePointsAttr([
        (-dx, -dy, -dz), 
        ( dx, -dy, -dz), 
        ( dx,  dy, -dz), 
        (-dx,  dy, -dz),        
        (-dx, -dy,  dz), 
        ( dx, -dy,  dz), 
        ( dx,  dy,  dz), 
        (-dx,  dy,  dz),
        ])
    box.CreateFaceVertexCountsAttr([4,4,4,4,4,4])
    box.CreateFaceVertexIndicesAttr([0, 3, 2, 1, 4, 5, 6, 7, 0, 1, 5, 4, 1, 2, 6, 5, 2, 3, 7, 6, 3, 0, 4, 7])
    box.CreateExtentAttr([(-dx, -dy, -dz), (dx, dy, dz)])
    box.CreateDisplayColorAttr([(0,0,1)])
    stage.Save()

In [6]:
n=0
for i,s in sh_d.items():
    if s.type=='UnidentifiedShape':
        print('skipping:', i, s.type, s.parameters)
        continue

    pars=s.parameters.split(';')
    par_d={}
    for p in pars:
        par_d[p.split('=')[0]]=float(p.split('=')[1])
    print(i, s.type, par_d)
    if s.type=='Box': 
        create_box(i, par_d)
        n+=1
        if n>10: break

1 Box {'XHalfLength': 10000.0, 'YHalfLength': 10000.0, 'ZHalfLength': 10000.0}
2 Tube {'RMin': 28.995, 'RMax': 245.0, 'ZHalfLength': 3460.0}
3 Tube {'RMin': 28.997, 'RMax': 143.0, 'ZHalfLength': 460.0}
4 Tube {'RMin': 29.757332898676882, 'RMax': 42.001, 'ZHalfLength': 339.101}
5 SimplePolygonBrep {'DZ': 339.101, 'NVertices': 10.0, 'xV': -1.5284006879314045, 'yV': -10.43}
6 Box {'XHalfLength': 0.39, 'YHalfLength': 10.43, 'ZHalfLength': 10.225}
7 Shift {'A': 6.0, 'X': 3.0}
8 Shift {'A': 7.0, 'X': 4.0}
9 Box {'XHalfLength': 0.6900000000000001, 'YHalfLength': 3.129, 'ZHalfLength': 10.225}
10 Shift {'A': 9.0, 'X': 5.0}
11 Union {'opA': 8.0, 'opB': 10.0}
12 Box {'XHalfLength': 0.345, 'YHalfLength': 3.129, 'ZHalfLength': 10.225}
13 Shift {'A': 12.0, 'X': 6.0}
14 Union {'opA': 11.0, 'opB': 13.0}
15 Box {'XHalfLength': 0.125, 'YHalfLength': 0.89, 'ZHalfLength': 10.235}
16 Shift {'A': 15.0, 'X': 7.0}
17 Union {'opA': 14.0, 'opB': 16.0}
18 Box {'XHalfLength': 0.04, 'YHalfLength': 4.71, 'ZHalfLeng